In [1]:
import os
import pandas as pd
import numpy as np
import random
from utils import from_np_array
import json
from monai.data.box_utils import convert_box_mode

In [2]:
content = pd.read_csv(
    '/data/output/msd_clean_nodules3.csv',
    converters={
        'spacing': from_np_array,
        'origin': from_np_array
    }
)
print(content.shape)
content.head()

(63, 10)


,name,spacing,origin,exam_refined,x_min,y_min,z_min,x_max,y_max,z_max
0,lung_001,"[1.25, 0.703125, 0.703125]","[-305.0, 40.15332031, 172.15332031]",False,371,255,185,392,273,199
1,lung_003,"[1.25, 0.703125, 0.703125]","[-377.75, 232.86250305, 226.1625061]",False,148,314,138,166,338,147
2,lung_004,"[1.25, 0.703125, 0.703125]","[-410.5, 211.171875, 215.2718811]",False,201,371,217,222,396,231
3,lung_005,"[1.25, 0.703125, 0.703125]","[-326.0, 0.172851562, 171.172852]",False,326,250,124,367,289,145
4,lung_006,"[1.25, 0.703125, 0.703125]","[-360.25, 229.02319336, 249.02319336]",False,277,259,224,312,294,244


### based on bounding box found, finding the center and diameter of the nodules

In [3]:
dataset = []
for hc_bb in content.itertuples(index=False):

    origin = np.array(list(reversed(hc_bb[2])))
    spacing = np.array(list(reversed(hc_bb[1])))

    voxel_min = np.array((hc_bb[4], hc_bb[5], hc_bb[6]))
    voxel_max = np.array((hc_bb[7], hc_bb[8], hc_bb[9]))

    voxel_coord_cent = voxel_min + (np.absolute(voxel_min - voxel_max)/2)
    
    voxel_coord_diam = voxel_max - voxel_min

    voxel_coord_cent[2] = voxel_coord_cent[2] * -1

    world_coord_cent = ((voxel_coord_cent * spacing) - origin ) * -1
    world_coord_diam = voxel_coord_diam * spacing
    
    dataset.append({
        "box": [[
                world_coord_cent[0],
                world_coord_cent[1],
                world_coord_cent[2],
                world_coord_diam[0],
                world_coord_diam[1],
                world_coord_diam[2]
            ]],
            "image": hc_bb[0] + '/' + hc_bb[0] + '.nii.gz',
            "label": [0]
    })

### data split 

In [4]:
random.shuffle(dataset)
print('total: ' + str(len(dataset)))
test_split_constant = 0.8
test_size = len(dataset) - int(len(dataset)*test_split_constant)
first_train_files, test_files = dataset[:-test_size], dataset[-test_size:]
print('train/val: ' + str(len(first_train_files)) + ' test: ' + str(len(test_files)))

save_file = open('/data/output/multi_data/msd_test.json', 'w')
value = { "test": test_files }
json.dump(value, save_file, indent=5)
save_file.close()

for fold_index in range(0, 10):
    dataset = first_train_files.copy()
    random.shuffle(dataset)
    
    val_split_constant = 0.8
    val_size = len(dataset) - int(len(dataset)*val_split_constant)
    train_files, val_files = dataset[:-val_size], dataset[-val_size:]

    if fold_index == 0:
        print('train: ' + str(len(train_files)) + ' val: ' + str(len(val_files)))

    save_file = open('/data/output/multi_data/msd_train_val' + str(fold_index) + '.json', 'w')
    value = {
        "training": train_files,
        "validation": val_files
    }
    json.dump(value, save_file, indent=5)
    save_file.close()

total: 63
train/val: 50 test: 13
train: 40 val: 10


### saving corresponding json for test

### saving corresponding json for train and val